<a href="https://colab.research.google.com/github/BraedynL0530/PortfolioWebsite/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
from dataclasses import  dataclass
torch.manual_seed(42)

#GPT-2 Example, resuse :3
class config:
  vocab_size: int #unique words
  block_size: int #how far back(context) it can see, memory/ how many tokens back
  n_layer: int = 12 # stacked blocks, more layers more reasoning more train time
  n_head: int = 12 # attentions per layer, how many "heads" looking for a new pattern
  n_embd: int = 768 #size of vector for each token
  dropout: float = 0.1  #prevents overfitting by stopping random paths


# Word token embedding, wte
# nn layer
vocab_size = 10
n_embed = 4


token_embedding_tabel = nn.Embedding(vocab_size, n_embed)

print("shape:", token_embedding_tabel.weight.shape)
print("weight:",token_embedding_tabel.weight)

# Word posetional embedding, wpe
"""
makes a unique vector for each poestion
vectors for each posetions, another nueral network layer
word and poestion exist in the same n_embd dimesion space
adding them creates a  unique point in the said space
learns that "the" in the beginning postion 1 isnt the same as "the" at posetion 12, distictions
"""


B, T, C = 2,5,n_embed #batch time/sequence length(tokens) channels(dimesions/n_embed)
block_size = 8
vocab_size = 10



poesition_embeding_tabel = nn.Embedding(block_size, C)

#input data
idx = torch.randint(0, vocab_size, (B,T))

#token emb data before
tok_emb = token_embedding_tabel(idx)

#after / poesitions for latest sequence length
pos = torch.arange(0, T, dtype=torch.long) #just counts up from 0 and says this is index 1 2 3... for each sequnce ex T=5 returns a tensor or ([0,1,2,3,4]) helps pos tabel look uip
pos_emb = poesition_embeding_tabel(pos)

x = tok_emb + pos_emb #makes a vector with both data ex tok=[1,5,8...] pos = [-.2, .3,-4...] x =[.8, 4.7, 4]


"""
nn.Embeddings create and store vectors,
torch.arrange tells what rows to return
batches are a single sequence, could be a set token ex 128,  a Readme for a project example, paragraph senante etc MY CHOICE AS TO WHAT, T is sequence length, blocksize is max sequnce
can go under but block_size is the limit
"""

print(token_embedding_tabel.weight.shape)
print(poesition_embeding_tabel.weight.shape)
print("final emb:",x)



shape: torch.Size([10, 4])
weight: Parameter containing:
tensor([[ 1.9269,  1.4873,  0.9007, -2.1055],
        [ 0.6784, -1.2345, -0.0431, -1.6047],
        [-0.7521,  1.6487, -0.3925, -1.4036],
        [-0.7279, -0.5594, -0.7688,  0.7624],
        [ 1.6423, -0.1596, -0.4974,  0.4396],
        [-0.7581,  1.0783,  0.8008,  1.6806],
        [ 0.0349,  0.3211,  1.5736, -0.8455],
        [ 1.3123,  0.6872, -1.0892, -0.3553],
        [-1.4181,  0.8963,  0.0499,  2.2667],
        [ 1.1790, -0.4345, -1.3864, -1.2862]], requires_grad=True)
torch.Size([10, 4])
torch.Size([8, 4])
final emb: tensor([[[-1.5953,  0.1559,  2.6121,  1.8412],
         [-0.3606, -0.3840,  0.6163,  0.3166],
         [ 0.1099,  1.3950, -2.1651,  0.1804],
         [ 2.4877,  1.2483, -1.5419, -1.1271],
         [-0.6128,  1.3094,  0.8095,  1.5383]],

        [[ 0.3418, -1.3568,  0.4250, -1.1257],
         [ 1.0457, -1.0591,  1.3421, -2.0505],
         [-1.9606,  1.7861, -0.2751,  2.2163],
         [ 1.8538, -0.6734, -0.495

In [ ]:
from torch.autograd import forward_ad
"""
self attention: part 1 of transformer
Q K V, query key value. helps use the two embeddings to learn diffrent meanings for words and give the diffrent vectors even if the same word
below is theory class is optimized, it condences the prjections into one huge vector and splits. other than that its nearly identical just more efficent
"""

"""
#learnable compenets
q_prog = nn.Linear(C, C, bias =False)
k_prog = nn.Linear(C, C, bias =False)
v_prog = nn.Linear(C, C, bias =False)

#weights
q_prog.weight.data = torch.randn(C,C)
q_prog.weight.data = torch.randn(C,C)
q_prog.weight.data = torch.randn(C,C)

#preform projection
q = q_prog(x)
k = k_prog(x)
v = v_prog(x)

scores = q @ k.transpose(-2,-1)
print("scores",scores)



Attention(Q,K,V)=softmax(​QK^⊤/dk​​)V

d_k = k.size(-1)#last dimesion of
scaled_scores = scores / math.sqrt(d_k)
attention_weights = F.softmax(scaled_scores, dim=1)
print("scaled scores", scaled_scores)
print("scaled scores -> percentages", attention_weights)

# aggreation Last part of attention!
output = attention_weights @ v
print("output!:",output)

"""

# Core logic for MultiHead
class CausualSelfAttention(nn.Module):
  def __init__(self, *args, **kwargs) -> None:
    super().__init__()
    assert self.config.n_embed % config.n_head == 0
    self.n_head = config.n_head
    self.n_embd = config.n_embd
    self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=False) # Fuzed layer = more efficent

    self.register_buffer( # part of causal masking
        "bias",# buffer name
        torch.trill(torch.ones(config.block_size,config.block_size))
        .view(1,1, config.block_size, config.block_size)
    )

    self.c_proj = nn.Linear(config.n_embd,config.n_embd)

  def forward(self, x):
    B, T, C = x.size()
    head_dim = C // self.n_head

    # project once -> split
    qkv = self.c_attn(x)
    q, k, v = qkv.split(C, dim=2)

    # reshape into heads
    q = q.view(B, T, self.n_head, head_dim).transpose(1, 2)
    k = k.view(B, T, self.n_head, head_dim).transpose(1, 2)
    v = v.view(B, T, self.n_head, head_dim).transpose(1, 2)

    # attention
    att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(head_dim))
    att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float("-inf")) # prevents it from seeing future tokens
    att = F.softmax(att, dim=-1)

    # aggregate :3
    y = att @ v

    # merge heads
    y = y.transpose(1, 2).contiguous().view(B, T, C)

    # final projection
    y = self.c_proj(y)
    return y